In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [0]:
import torch

cuda = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np

totalbars=2 #total bars as input
NUM_PITCHES=4 # all possible notes to play


NOTESPERBAR=16
TOTAL_NOTES=NOTESPERBAR*totalbars

num_features=NUM_PITCHES #size of input feature vector

teacherforcing=False #not used but it will be needed


In [3]:

#Generating dummy data
a= np.random.randint(NUM_PITCHES, size=TOTAL_NOTES)
print(a)
data = np.zeros((TOTAL_NOTES, NUM_PITCHES))
data[np.arange(TOTAL_NOTES), a] = 1 #generating dummy data
data

[0 1 0 2 2 3 0 1 1 2 2 1 0 0 2 2 0 2 1 2 3 2 2 0 2 1 0 1 1 3 1 2]


array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

In [0]:
#half of this is not yet needed but maybe it will be to visualize the latent space
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
from sklearn.decomposition import PCA
%matplotlib nbagg
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
sns.set_palette(sns.dark_palette("purple"))

MODEL:

In [7]:
import torch.nn as nn
from torch.nn.functional import softplus

# define size variables
input_size = NUM_PITCHES

enc_hidden_size=2048 #hidden size of encoder
dec_hidden_size=2048 #hidden size of decoder
latent_features=512 #latent space dimension



class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_features):
        super(VariationalAutoencoder, self).__init__()
        
        self.latent_features = latent_features
       
        
        
        #data goes into bidirectional encoder
        self.encoder = torch.nn.LSTM(
                batch_first = True,
                input_size = input_size,
                hidden_size = enc_hidden_size,
                num_layers = 1,
                bidirectional = True)
        
        #encoded data goes onto connect linear layer. inputs must be*2 because LSTM is bidirectional
        #output must be 2*latentspace because it needs to be split into miu and sigma right after.
        self.encoderOut = nn.Linear(in_features=enc_hidden_size*2, out_features=latent_features*2)
        
        
        #Model after this line has not been tested -------------
        self.decoderIn = nn.Linear(in_features=latent_features, out_features=dec_hidden_size)
        
        # The latent code must be decoded into the original image
        self.decoder = torch.nn.LSTM(
                batch_first=True,
                input_size=input_size,
                hidden_size=dec_hidden_size,
                num_layers=3,
                bidirectional = False)

    #used to initialize the hidden layer of the encoder to zero before every batch
    def init_hidden(self, batch_size):
      #must be 2 x batch x hidden_size because its a bi-directional LSTM
      init = torch.zeros(2, batch_size, enc_hidden_size, device=device)
      c0 = torch.zeros(2, batch_size, enc_hidden_size, device=device)
      return init,c0


    def forward(self, x): 
        outputs = {}
        
        print("INPUT SIZE",x.size())
        
        #creates hidden layer values
        batch_size = x.shape[0]
        h0,c0 = self.init_hidden(batch_size)
        
        #resets encoder at the beginning of every batch
        x, hidden = self.encoder(x, ( h0,c0))
          
        print("after rnn SIZE",x.shape)
        
        #goes from 4096 to 1024
        x = self.encoderOut(x)
        
        print("after linear",x.shape)
        
        # Split encoder outputs into a mean and variance vector
        mu, log_var = torch.chunk(x, 2, dim=-1)
        
        print("MU SIZE",mu.shape)
        print("LOG SIZE",log_var.shape)
        
        # Make sure that the log variance is positive
        log_var = softplus(log_var)
        
        # :- Reparametrisation trick
        # a sample from N(mu, sigma) is mu + sigma * epsilon
        # where epsilon ~ N(0, 1)
                
        # Don't propagate gradients through randomness
        with torch.no_grad():
            batch_size = mu.size(0)
            epsilon = torch.randn(batch_size, 1, self.latent_features)
            
            if cuda:
                epsilon = epsilon.cuda()
        
        #setting sigma
        sigma = torch.exp(log_var*2)
        
        
        print("ready to make latent space",sigma.shape)
        
        #generate z
        z = mu + epsilon * sigma        
        
        
        #NOT DONE AFTER THIS
        
        # Run through decoder
        x = self.decoder(z)
        
        # The original digits are on the scale [0, 1]
        x = torch.sigmoid(x)
        
        # Mean over samples
        x_hat = torch.mean(x, dim=1)
        
        outputs["x_hat"] = x_hat
        outputs["z"] = z
        outputs["mu"] = mu
        outputs["log_var"] = log_var
        
        return outputs




net = VariationalAutoencoder(latent_features)

# Transfer model to GPU if available
if cuda:
    net = net.cuda()

print(net)

VariationalAutoencoder(
  (encoder): LSTM(4, 2048, batch_first=True, bidirectional=True)
  (encoderOut): Linear(in_features=4096, out_features=1024, bias=True)
  (decoderIn): Linear(in_features=512, out_features=2048, bias=True)
  (decoder): LSTM(4, 2048, num_layers=3, batch_first=True)
)


Testing if forward pass works

In [8]:
from torch.autograd import Variable

#setting dummy data
x = data

#adding 1 dimension, 1x32x4
y = x[np.newaxis, :, :]

#setting input ans tensor variable
x = Variable(torch.Tensor(y))

if cuda:
    x = x.cuda()

#running forward pass
outputs = net(x)


#AFTER THIS NOTHING IS DONE YET ------
x_hat = outputs["x_hat"]
mu, log_var = outputs["mu"], outputs["log_var"]
z = outputs["z"]

loss, kl = loss_function(x_hat, x, mu, log_var)

print(x.shape)
print(x_hat.shape)
print(z.shape)
print(loss)



                
                
print(kl)

INPUT SIZE torch.Size([1, 32, 4])
after rnn SIZE torch.Size([1, 32, 4096])
latentspace 512
after linear torch.Size([1, 32, 1024])
MU SIZE torch.Size([1, 32, 512])
LOG SIZE torch.Size([1, 32, 512])
ready to make torch.Size([1, 32, 512])


RuntimeError: ignored

Optimizer

In [0]:
#directly taken from notebook, probably some adaptation might be needed

from torch.nn.functional import binary_cross_entropy
from torch import optim

def ELBO_loss(y, t, mu, log_var):
    # Reconstruction error, log[p(x|z)]
    # Sum over features
    likelihood = -binary_cross_entropy(y, t, reduction="none")
    likelihood = likelihood.view(likelihood.size(0), -1).sum(1)

    # Regularization error: 
    # Kulback-Leibler divergence between approximate posterior, q(z|x)
    # and prior p(z) = N(z | mu, sigma*I).
    
    # In the case of the KL-divergence between diagonal covariance Gaussian and 
    # a standard Gaussian, an analytic solution exists. Using this excerts a lower
    # variance estimator of KL(q||p)
    kl = -0.5 * torch.sum(1 + log_var - mu**2 - torch.exp(log_var), dim=1)

    # Combining the two terms in the evidence lower bound objective (ELBO) 
    # mean over batch
    ELBO = torch.mean(likelihood) - torch.mean(kl)
    
    # notice minus sign as we want to maximise ELBO
    return -ELBO, kl.sum()


# define our optimizer
# The Adam optimizer works really well with VAEs.
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = ELBO_loss